<a href="https://colab.research.google.com/github/adi180397/Social-media-claim-detection/blob/main/Social_media_claim_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer

In [5]:
df=pd.read_csv('/content/train.csv')

In [6]:
df.drop(columns=['ID'],inplace=True)

In [7]:
#Converting the text into lowercase
df['tweet_text_lower']=df['tweet_text'].str.lower()

In [8]:
df.drop(columns=['tweet_text'],inplace=True)


In [9]:
import nltk

In [10]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [13]:
df.head()

,claim,tweet_text_lower
0,1,coronavirus may have originated in lab linked ...
1,1,@scmpnews china will buy all shares that's y i...
2,1,i'm curious if the pneumonia vaccine (prevnar-...
3,1,dear you knew about covid 19 in january it is ...
4,1,"wow, they're as dumb as @realdonaldtrump sugge..."


In [14]:
def expand_contractions(text):
    contractions_dict = {
        "ain't": "is not",
        "aren't": "are not",
        "can't": "cannot",
        "can't've": "cannot have",
        "'cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he would",
        "he'd've": "he would have",
        "he'll": "he will",
        "he'll've": "he will have",
        "he's": "he is",
        "how'd": "how did",
        "how'd'y": "how do you",
        "how'll": "how will",
        "how's": "how is",
        "I'd": "I would",
        "I'd've": "I would have",
        "I'll": "I will",
        "I'll've": "I will have",
        "I'm": "I am",
        "I've": "I have",
        "i'd": "i would",
        "i'd've": "i would have",
        "i'll": "i will",
        "i'll've": "i will have",
        "i'm": "i am",
        "i've": "i have",
        "isn't": "is not",
        "it'd": "it would",
        "it'd've": "it would have",
        "it'll": "it will",
        "it'll've": "it will have",
        "it's": "it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "must have",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "o'clock": "of the clock",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "she would",
        "she'd've": "she would have",
        "she'll": "she will",
        "she'll've": "she will have",
        "she's": "she is",
        "should've": "should have",
        "shouldn't": "should not",
        "shouldn't've": "should not have",
        "so've": "so have",
        "so's": "so as",
        "that'd": "that would",
        "that'd've": "that would have",
        "that's": "that is",
        "there'd": "there would",
        "there'd've": "there would have",
        "there's": "there is",
        "they'd": "they would",
        "they'd've": "they would have",
        "they'll": "they will",
        "they'll've": "they will have",
        "they're": "they are",
        "they've": "they have",
        "to've": "to have",
        "wasn't": "was not",
        "we'd": "we would",
        "we'd've": "we would have",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what will",
        "what'll've": "what will have",
        "what're": "what are",
        "what's": "what is",
        "what've": "what have",
        "when's": "when is",
        "when've": "when have",
        "where'd": "where did",
        "where's": "where is",
        "where've": "where have",
        "who'll": "who will",
        "who'll've": "who will have",
        "who's": "who is",
        "who've": "who have",
        "why's": "why is",
        "why've": "why have",
        "will've": "will have",
        "won't": "will not",
        "won't've": "will not have",
        "would've": "would have",
        "wouldn't": "would not",
        "wouldn't've": "would not have",
        "y'all": "you all",
        "y'all'd": "you all would",
        "y'all'd've": "you all would have",
        "y'all're": "you all are",
        "y'all've": "you all have",
        "you'd": "you would",
        "you'd've": "you would have",
        "you'll": "you will",
        "you'll've": "you will have",
        "you're": "you are",
        "you've": "you have",
        "i'm":"i am"
    }

    # Create a regular expression pattern that matches any of the contractions
    contractions_pattern = re.compile('({})'.format('|'.join(contractions_dict.keys())),
                                      flags=re.IGNORECASE|re.DOTALL)

    # Replace contractions with their expanded forms
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contractions_dict.get(match) \
                                if contractions_dict.get(match) \
                                else contractions_dict.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    return expanded_text

In [15]:
df['text_con']=df['tweet_text_lower'].apply(lambda text: expand_contractions(text))


In [16]:
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)

    # Remove mentions (e.g., @username)
    text = re.sub(r'@\w+', '', text)

    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    return text

In [17]:
df['text_cleaned']=df['text_con'].apply(lambda text: clean_text(text))


In [18]:
df.head()

,claim,tweet_text_lower,text_con,text_cleaned
0,1,coronavirus may have originated in lab linked ...,coronavirus may have originated in lab linked ...,coronavirus may have originated in lab linked ...
1,1,@scmpnews china will buy all shares that's y i...,@scmpnews china will buy all shares that is y ...,china will buy all shares that is y it create...
2,1,i'm curious if the pneumonia vaccine (prevnar-...,i am curious if the pneumonia vaccine (prevnar...,i am curious if the pneumonia vaccine prevnar ...
3,1,dear you knew about covid 19 in january it is ...,dear you knew about covid 19 in january it is ...,dear you knew about covid in january it is ma...
4,1,"wow, they're as dumb as @realdonaldtrump sugge...","wow, they are as dumb as @realdonaldtrump sugg...",wow they are as dumb as suggesting that injec...


In [19]:
def tokenize(text):
    return word_tokenize(text)

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return filtered_tokens

def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return lemmatized_tokens
def stem(tokens):
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return stemmed_tokens

In [20]:
df['text_token']=df['text_cleaned'].apply(lambda text: tokenize(text))



In [21]:
 df['text_stop']=df['text_token'].apply(lambda tokens: remove_stopwords(tokens))


In [22]:
df['text_lemi']=df['text_stop'].apply(lambda tokens: lemmatize(tokens))

In [23]:
df['text_stem']=df['text_lemi'].apply(lambda tokens: stem(tokens))

In [24]:
df.head()

,claim,tweet_text_lower,text_con,text_cleaned,text_token,text_stop,text_lemi,text_stem
0,1,coronavirus may have originated in lab linked ...,coronavirus may have originated in lab linked ...,coronavirus may have originated in lab linked ...,"[coronavirus, may, have, originated, in, lab, ...","[coronavirus, may, originated, lab, linked, ch...","[coronavirus, may, originated, lab, linked, ch...","[coronaviru, may, origin, lab, link, china, bi..."
1,1,@scmpnews china will buy all shares that's y i...,@scmpnews china will buy all shares that is y ...,china will buy all shares that is y it create...,"[china, will, buy, all, shares, that, is, y, i...","[china, buy, shares, created, rna, virus, muta...","[china, buy, share, created, rna, virus, mutat...","[china, buy, share, creat, rna, viru, mutat, c..."
2,1,i'm curious if the pneumonia vaccine (prevnar-...,i am curious if the pneumonia vaccine (prevnar...,i am curious if the pneumonia vaccine prevnar ...,"[i, am, curious, if, the, pneumonia, vaccine, ...","[curious, pneumonia, vaccine, prevnar, would, ...","[curious, pneumonia, vaccine, prevnar, would, ...","[curiou, pneumonia, vaccin, prevnar, would, be..."
3,1,dear you knew about covid 19 in january it is ...,dear you knew about covid 19 in january it is ...,dear you knew about covid in january it is ma...,"[dear, you, knew, about, covid, in, january, i...","[dear, knew, covid, january, march, administra...","[dear, knew, covid, january, march, administra...","[dear, knew, covid, januari, march, administr,..."
4,1,"wow, they're as dumb as @realdonaldtrump sugge...","wow, they are as dumb as @realdonaldtrump sugg...",wow they are as dumb as suggesting that injec...,"[wow, they, are, as, dumb, as, suggesting, tha...","[wow, dumb, suggesting, injecting, disinfectan...","[wow, dumb, suggesting, injecting, disinfectan...","[wow, dumb, suggest, inject, disinfect, cure, ..."


In [25]:
from transformers import GPT2Tokenizer, GPT2Model
import torch


In [26]:
# Load pre-trained GPT tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [27]:
def get_gpt_embeddings(text):
    # Check if text is empty
    if not text:
        return None  # or you can return a default value
    else:
        input_ids = tokenizer.encode(text, return_tensors='pt')
        with torch.no_grad():
            outputs = model(input_ids)
            embeddings = outputs.last_hidden_state
        return embeddings



In [28]:
# Apply the function to the 'text_lemi' column in the DataFrame after filtering out empty texts
df['gpt_embeddings'] = df['text_lemi'].apply(lambda x: get_gpt_embeddings(x) if x else None)



In [29]:
df.head()

,claim,tweet_text_lower,text_con,text_cleaned,text_token,text_stop,text_lemi,text_stem,gpt_embeddings
0,1,coronavirus may have originated in lab linked ...,coronavirus may have originated in lab linked ...,coronavirus may have originated in lab linked ...,"[coronavirus, may, have, originated, in, lab, ...","[coronavirus, may, originated, lab, linked, ch...","[coronavirus, may, originated, lab, linked, ch...","[coronaviru, may, origin, lab, link, china, bi...","[[[tensor(-0.0667), tensor(0.0881), tensor(-0...."
1,1,@scmpnews china will buy all shares that's y i...,@scmpnews china will buy all shares that is y ...,china will buy all shares that is y it create...,"[china, will, buy, all, shares, that, is, y, i...","[china, buy, shares, created, rna, virus, muta...","[china, buy, share, created, rna, virus, mutat...","[china, buy, share, creat, rna, viru, mutat, c...","[[[tensor(-0.0667), tensor(0.0881), tensor(-0...."
2,1,i'm curious if the pneumonia vaccine (prevnar-...,i am curious if the pneumonia vaccine (prevnar...,i am curious if the pneumonia vaccine prevnar ...,"[i, am, curious, if, the, pneumonia, vaccine, ...","[curious, pneumonia, vaccine, prevnar, would, ...","[curious, pneumonia, vaccine, prevnar, would, ...","[curiou, pneumonia, vaccin, prevnar, would, be...","[[[tensor(-0.0667), tensor(0.0881), tensor(-0...."
3,1,dear you knew about covid 19 in january it is ...,dear you knew about covid 19 in january it is ...,dear you knew about covid in january it is ma...,"[dear, you, knew, about, covid, in, january, i...","[dear, knew, covid, january, march, administra...","[dear, knew, covid, january, march, administra...","[dear, knew, covid, januari, march, administr,...","[[[tensor(-0.0667), tensor(0.0881), tensor(-0...."
4,1,"wow, they're as dumb as @realdonaldtrump sugge...","wow, they are as dumb as @realdonaldtrump sugg...",wow they are as dumb as suggesting that injec...,"[wow, they, are, as, dumb, as, suggesting, tha...","[wow, dumb, suggesting, injecting, disinfectan...","[wow, dumb, suggesting, injecting, disinfectan...","[wow, dumb, suggest, inject, disinfect, cure, ...","[[[tensor(-0.0189), tensor(-0.0223), tensor(-0..."


In [30]:
new_df=df[['gpt_embeddings','claim']]

In [31]:
new_df.head(2)

,gpt_embeddings,claim
0,"[[[tensor(-0.0667), tensor(0.0881), tensor(-0....",1
1,"[[[tensor(-0.0667), tensor(0.0881), tensor(-0....",1


In [32]:
X_train=df['gpt_embeddings']

In [33]:
y_train=df['claim']

In [34]:
df_dev=pd.read_csv('/content/dev.csv')

In [35]:
df_dev.head()

,ID,tweet_text,claim
0,5580,friends the stuff about nature healing and wil...,1
1,8167,A9. If you’re thinking of taking action based ...,1
2,160,"Kyle Turley, co-owner of Moreno Valley dispens...",1
3,5196,exhibit a,0
4,3392,The English edition of Marathi daily Lokmat pu...,1


In [36]:
df_dev.drop(columns=['ID'],inplace=True)

In [37]:
df_dev.head(2)

,tweet_text,claim
0,friends the stuff about nature healing and wil...,1
1,A9. If you’re thinking of taking action based ...,1


In [38]:
df_dev['tweet_text_lower']=df_dev['tweet_text'].str.lower()

In [39]:
df_dev.head(2)

,tweet_text,claim,tweet_text_lower
0,friends the stuff about nature healing and wil...,1,friends the stuff about nature healing and wil...
1,A9. If you’re thinking of taking action based ...,1,a9. if you’re thinking of taking action based ...


In [40]:
df_dev.drop(columns=['tweet_text'],inplace=True)


In [41]:
df_dev['text_con']=df_dev['tweet_text_lower'].apply(lambda text: expand_contractions(text))


In [42]:
df_dev['text_cleaned']=df_dev['text_con'].apply(lambda text: clean_text(text))
df_dev['text_token']=df_dev['text_cleaned'].apply(lambda text: tokenize(text))

df_dev['text_stop']=df_dev['text_token'].apply(lambda tokens: remove_stopwords(tokens))
df_dev['text_lemi']=df_dev['text_stop'].apply(lambda tokens: lemmatize(tokens))

In [43]:
df_dev.head(3)

,claim,tweet_text_lower,text_con,text_cleaned,text_token,text_stop,text_lemi
0,1,friends the stuff about nature healing and wil...,friends the stuff about nature healing and wil...,friends the stuff about nature healing and wil...,"[friends, the, stuff, about, nature, healing, ...","[friends, stuff, nature, healing, wildlife, re...","[friend, stuff, nature, healing, wildlife, ret..."
1,1,a9. if you’re thinking of taking action based ...,a9. if you’re thinking of taking action based ...,a if youre thinking of taking action based on ...,"[a, if, youre, thinking, of, taking, action, b...","[youre, thinking, taking, action, based, somet...","[youre, thinking, taking, action, based, somet..."
2,1,"kyle turley, co-owner of moreno valley dispens...","kyle turley, co-owner of moreno valley dispens...",kyle turley coowner of moreno valley dispensar...,"[kyle, turley, coowner, of, moreno, valley, di...","[kyle, turley, coowner, moreno, valley, dispen...","[kyle, turley, coowner, moreno, valley, dispen..."


In [44]:
df_dev['gpt_embeddings'] = df_dev['text_lemi'].apply(lambda x: get_gpt_embeddings(x) if x else None)


In [45]:
X_dev=df_dev['gpt_embeddings']
y_dev=df_dev['claim']

In [46]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [47]:
X_train

0       [[[tensor(-0.0667), tensor(0.0881), tensor(-0....
1       [[[tensor(-0.0667), tensor(0.0881), tensor(-0....
2       [[[tensor(-0.0667), tensor(0.0881), tensor(-0....
3       [[[tensor(-0.0667), tensor(0.0881), tensor(-0....
4       [[[tensor(-0.0189), tensor(-0.0223), tensor(-0...
                              ...                        
6981    [[[tensor(0.0071), tensor(0.0532), tensor(-0.3...
6982    [[[tensor(-0.0667), tensor(0.0881), tensor(-0....
6983    [[[tensor(-0.0667), tensor(0.0881), tensor(-0....
6984    [[[tensor(-0.0932), tensor(0.0294), tensor(-0....
6985    [[[tensor(-0.0667), tensor(0.0881), tensor(-0....
Name: gpt_embeddings, Length: 6986, dtype: object

In [56]:
X_train_np = X_train.to_numpy()
X_dev_np = X_dev.to_numpy()

In [57]:
X_train_np

array([tensor([[[-0.0667,  0.0881, -0.3085,  ...,  0.0307,  0.0512, -0.0019],
                [-0.9246, -0.4948, -0.2018,  ..., -0.0020,  0.2230,  0.0590],
                [ 0.0189, -0.0236,  0.0827,  ...,  0.1012, -0.0799,  0.0131],
                ...,
                [-0.0798,  0.1678, -0.1790,  ..., -0.1693, -0.0371, -0.2302],
                [ 0.2461,  0.3430, -0.0749,  ...,  0.0729,  0.2845, -0.0326],
                [ 0.0845,  0.0102,  0.2646,  ...,  0.2003, -0.1288, -0.0856]]]),
       tensor([[[-0.0667,  0.0881, -0.3085,  ...,  0.0307,  0.0512, -0.0019],
                [ 0.9210, -0.7846, -1.5334,  ..., -0.6707,  0.3358, -0.1350],
                [ 1.4065,  0.3304, -0.7031,  ...,  0.2430,  0.3852,  0.1007],
                ...,
                [ 0.1367,  0.1711, -0.4736,  ...,  0.1241,  0.2283, -0.2883],
                [-0.0217,  0.0889,  0.2269,  ...,  0.1209, -0.1511, -0.1298],
                [-0.0045,  0.0678,  0.1879,  ...,  0.1304, -0.1359, -0.1036]]]),
       tensor([[

In [53]:
X_train_np

tensor([[[-0.1517, -0.0071, -0.3982,  ..., -0.1050,  0.0251, -0.1273],
         [ 0.1637, -0.2658, -0.0637,  ..., -0.4001,  0.1480,  0.1684],
         [ 0.0986, -0.0236, -0.0562,  ..., -0.4225,  0.3047,  0.4919],
         ...,
         [ 0.0308, -0.0227,  0.1067,  ...,  0.1081, -0.1020,  0.0149],
         [ 0.0320, -0.0058,  0.1211,  ...,  0.1148, -0.1057,  0.0133],
         [ 0.5327,  0.3092,  0.3080,  ..., -0.5195,  0.7390, -0.3422]]])

In [63]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Define a function to pad sequences with zeros
def pad_sequence(sequence, max_length):
    if sequence is None:
        return np.zeros(max_length)  # Return an array of zeros if sequence is None
    else:
        sequence_flat = sequence.numpy().flatten()[:max_length]  # Flatten and truncate sequence
        padded_sequence = np.pad(sequence_flat, (0, max_length - len(sequence_flat)), mode='constant')  # Pad with zeros
        return padded_sequence

# Set the maximum length for padding
max_length = 300  # Set your desired maximum length here

# Pad sequences in X_train_np and X_dev_np with zeros
X_train_padded = np.array([pad_sequence(sequence, max_length) for sequence in X_train_np])
X_dev_padded = np.array([pad_sequence(sequence, max_length) for sequence in X_dev_np])

# Create RandomForestClassifier instance
rf_classifier = RandomForestClassifier()

# Train the classifier
rf_classifier.fit(X_train_padded, y_train)

# Make predictions on the dev data
y_pred_dev = rf_classifier.predict(X_dev_padded)

# Evaluate the model
accuracy_dev = accuracy_score(y_dev, y_pred_dev)
print("Dev Accuracy:", accuracy_dev)


Dev Accuracy: 0.8603874415497662
